## Learning to chose the suit to lead on (pass) - 1nt - all pass
to add: treat the same hands with differents leads; check the dependence of 1nt force (!need to calculate points of declarer)

Script to get the leading hand and the lead from file "temp_final.DAT" which containts hands + leads

In [31]:
#you gotta take out the newline in the beginning of the file
import numpy as np

PLAYERS = {0:'E', 1:'S', 2:'W', 3:'N'}
SUITMAP = {'S':0, 'H':1, 'D':2, 'C':3}
##added by me
SUITS = ['S', 'H', 'D', 'C', 'N'] ## N for no out of range exception
##
CARDMAP = {'2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, 'T':10, 'J':11, 'Q':12, 'K':13, 'A':14}

##starts to read from the first S and returns the strings for hands and entame
def pretreat(horribleInput):
    horribleInput = horribleInput.upper()
    
    begin = horribleInput.find(SUITS[0]) ##beginning of hands, namely the first spades that he encounters
    if(begin == -1):
        print("couldnt find 'S' in the horrible input");
    args = horribleInput[begin:].split('|')
    if(len(args) != 2):
        print("there was more than one '|' after the first S(spades)")
    if(len(args[0]) != 71):
        print("the collection of hands(17 chars in each and 3 commas) has a weird length:" + len(args[0]))
    if(len(args[1]) != 2):
        print("entame with plus than 2 chars")
    return args[0], args[1]

#magic! returns vector from text format hand
def vectorizing(handText):
    hand = np.zeros(53)
    end = 0   #should see if its S
    for suit in range(4):
        begin = end + 1
        end = handText.find(SUITS[suit+1])
        for i in range(begin,end):
            hand[suit*13 + CARDMAP[handText[i]]-2] = 1
    return hand
        
# colocar um nome melhor
#return who entamed, in terms of S, W, E, N, with the players hands in the orther they`ll be played
def treatingDudu(horribleInput):
    #handsText com in the format [13],...
    #uniformilizing the inputs
    handsText, entame = pretreat(horribleInput)
    #print(handsText)
    #print(entame)
    handsText = handsText.split(',')
    suitEntame = entame[0]
    rankEntame = entame[1]
    suitCode = SUITMAP[suitEntame]
    nextSuit = SUITS[suitCode + 1]
    #hands = np.zeros((4, 53))
    #print(suitEntame, nextSuit)
    hands = []
    for h in range(len(handsText)):
        
        begin = handsText[h].index(suitEntame)
        end = handsText[h].find(nextSuit)
        #print(end)
        if(end == -1):
            end = len(handsText[h])
        if(handsText[h][begin:end].find(rankEntame) != -1):
            found = h
            #print(h)
            break
    for h in range(len(handsText)):
        hands.append(vectorizing(handsText[(found+h)%len(handsText)]))
        
    
    #print(found)
    #print(PLAYERS[found])
    
    return entame, hands[0] ##,PLAYERS[found]

#appele treating pour une 'file'
def archiveTreat(filename):
    terribleArchive = open(filename, 'r')
    horribleInputs = terribleArchive.read().split('\n')
    return [treatingDudu(inp) for inp in horribleInputs]
    
#horribleInput = "SAK62H32DAKT6CA84,SJT7HAJ4DQJ982C63,S83HT876D75CKQ752,SQ954HKQ95D43CJT9|DQ"
#print(treatingDudu(horribleInput))
filename = "temp_final.DAT"
vectors = archiveTreat(filename)

Creating two arrays with hands and corresponding leads

In [32]:
leads = []
hands = []
length = len(vectors)
for i in range(length):
    leads.append(vectors[i][0])
    hands.append(vectors[i][1][:-1])
#S = 0, H = 1, D = 2, C = 3
suits_lead = []
for i in range(length):
    if leads[i][0] == 'S':
        suits_lead.append(0)
    if leads[i][0] == 'H':
        suits_lead.append(1)
    if leads[i][0] == 'D':
        suits_lead.append(2)
    if leads[i][0] == 'C':
        suits_lead.append(3)

Making a classifier

In [35]:
#import pandas as pd
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
#columns = ['2s', '3s', '4s', '5s','6s', '7s', '8s', '9s','10s', 'Js', 'Qs', 'Ks', 'As', 
#           '2h', '3h', '4h', '5h','6h', '7h', '8h', '9h','10h', 'Jh', 'Qh', 'Kh', 'Ah',
#           '2d', '3d', '4d', '5d','6d', '7d', '8d', '9d','10d', 'Jd', 'Qd', 'Kd', 'Ad',
#           '2c', '3c', '4c', '5c','6c', '7c', '8c', '9c','10c', 'Jc', 'Qc', 'Kc', 'Ac']
#df = pd.DataFrame(hands, columns=columns)

Learning and making a train_test_split

In [36]:
X_train, X_test, y_train, y_test = train_test_split(hands, suits_lead, test_size=0.1, random_state=0)
clf = OneVsRestClassifier(SVC(random_state=0)).fit(X_train, y_train)
clf.score(X_test, y_test)

0.67272727272727273

Increasing number of boards by swipping majors and minors

In [37]:
#exchange majors; exchange minors; exchange both
for i in range(length):
    
    #exchange majors
    new_hand = np.zeros(52)
    new_hand[:13], new_hand[13:26], new_hand[26:52] = hands[i][13:26], hands[i][0:13], hands[i][26:52]
    hands.append(new_hand)
    if suits_lead[i] == 0:
        suits_lead.append(1)
    if suits_lead[i] == 1:
        suits_lead.append(0)
    if suits_lead[i] == 2:
        suits_lead.append(2)
    if suits_lead[i] == 3:
        suits_lead.append(3)
        
    #exchange minors
    new_hand = np.zeros(52)
    new_hand[:26], new_hand[26:39], new_hand[39:52] = hands[i][0:26], hands[i][39:52], hands[i][26:39]
    hands.append(new_hand)
    if suits_lead[i] == 0:
        suits_lead.append(0)
    if suits_lead[i] == 1:
        suits_lead.append(1)
    if suits_lead[i] == 2:
        suits_lead.append(3)
    if suits_lead[i] == 3:
        suits_lead.append(2)
        
    #exchange both
    new_hand = np.zeros(52)
    new_hand[:13], new_hand[13:26], new_hand[26:39], new_hand[39:52] = hands[i][13:26], hands[i][0:13], hands[i][39:52], hands[i][26:39]
    hands.append(new_hand)
    if suits_lead[i] == 0:
        suits_lead.append(1)
    if suits_lead[i] == 1:
        suits_lead.append(0)
    if suits_lead[i] == 2:
        suits_lead.append(3)
    if suits_lead[i] == 3:
        suits_lead.append(2)

Learning on bigger dataset and making a train_test_split

In [41]:
X_train, X_test, y_train, y_test = train_test_split(hands, suits_lead, test_size=0.1, random_state=0)
clf = OneVsRestClassifier(SVC(random_state=0)).fit(X_train, y_train)
clf.score(X_test, y_test)

0.67727272727272725